<a href="https://colab.research.google.com/github/ExCaLBBR/ExCaLBBR_Projects/blob/main/RaciallyBiasedDecisions/Racialized-Gendered_Names/analysis/RGN_proportions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Load relevant libraries
import pandas as pd

In [27]:
#User specification here
  #Data dir
fil = 'https://raw.githubusercontent.com/ExCaLBBR/ExCaLBBR_Projects/main/RaciallyBiasedDecisions/Racialized-Gendered_Names/data/taskRGN.csv'

  #SpecRemove specific participants
   #Current condition: fixed responded & incorrect response to check
rem = ['dwgxuk', 'tdqdqk', 'ppluks']

In [70]:
#Load data
df_task = pd.read_csv(fil, index_col = 0, header = [0,1,2])
#display(df_task)

#Parse relevant substructures
df_RaceLabels = df_task['Response']['Race']
df_GenderLabels = df_task['Response']['Gender']
df_RaceRT = df_task['Reaction Time']['Gender']
df_GenderRT = df_task['Reaction Time']['Gender']

#Remove specified participants
remIdxRL = [df_RaceLabels.columns.get_loc(r) for r in rem if r in df_RaceLabels]
remIdxGL = [df_GenderLabels.columns.get_loc(r) for r in rem if r in df_GenderLabels]
remIdxRT = [df_RaceRT.columns.get_loc(r) for r in rem if r in df_RaceRT]
remIdxGT = [df_GenderRT.columns.get_loc(r) for r in rem if r in df_GenderRT]

df_RaceLabels = df_RaceLabels.drop(df_RaceLabels.columns[remIdxRL],axis = 1)
df_GenderLabels  = df_GenderLabels.drop(df_GenderLabels.columns[remIdxGL],axis = 1)
df_RaceRT  = df_RaceRT.drop(df_RaceRT.columns[remIdxRT],axis = 1)
df_GenderRT  = df_GenderRT.drop(df_GenderRT.columns[remIdxGT],axis = 1)

In [69]:
display(df_RaceLabels)


Participant Completion Code,vcdueq,lpuvnk,dhyrks,uvoope,ximjis,cccpda,kyxjzg,dwgxuk,ofgxam,yutsau,...,xttste,vvymza,fffyhp,cxiely,raebyp,agqsng,oaqtmi,aejujn,illyaj,xesqkj
# name,,,,,,,,,,,,,,,,,,,,,
# Aaliyiah,Hispanic or Latino,White American,Black American,Black American,Black American,Black American,Black American,White American,Black American,Black American,...,Black American,Black American,Black American,Black American,Black American,Black American,Black American,Black American,Black American,Black American
# Aisha,Black American,Black American,Black American,Black American,Black American,Black American,Black American,White American,White American,Black American,...,Black American,Black American,Black American,Black American,Black American,Black American,Black American,Black American,Black American,Black American
# Alex,White American,White American,White American,White American,White American,White American,Hispanic or Latino,White American,White American,White American,...,White American,White American,White American,White American,White American,White American,White American,White American,Hispanic or Latino,White American
# Becky,White American,White American,White American,White American,Black American,White American,White American,White American,White American,White American,...,White American,Hispanic or Latino,White American,White American,White American,White American,White American,White American,White American,White American
# Brody,White American,White American,White American,White American,Black American,White American,White American,White American,White American,White American,...,White American,White American,Black American,White American,White American,White American,White American,White American,White American,White American
# Casey,White American,White American,White American,White American,Chinese American,White American,Black American,White American,White American,White American,...,White American,White American,White American,White American,White American,White American,White American,White American,White American,White American
# Charlotte,White American,White American,White American,White American,White American,White American,Black American,White American,White American,White American,...,White American,White American,White American,White American,White American,White American,White American,White American,White American,White American
# Deshawn,Black American,Black American,Black American,Black American,Black American,White American,Black American,White American,Black American,Black American,...,Black American,Black American,Black American,Black American,Black American,Black American,Black American,Black American,Black American,Black American
# Emma,White American,White American,White American,White American,Chinese American,White American,White American,White American,White American,White American,...,White American,White American,Black American,White American,White American,Black American,White American,White American,White American,White American


In [4]:
%ls